In [1]:
!python --version

Python 3.7.3


# Ensuring Concating Script works locally

In [4]:
import pandas as pd
import os
import sys
import json
import pyarrow.feather as feather

#import movement.config as CONFIG

data_path ='C:\\Users\\Anthony\\Documents\\CapstoneDS-5999\\temp_data'
#feather_path = '/data/p_dsi/nba_player_movement/concat_dataframe'
files = os.listdir(data_path)

#if not os.path.exists(feather_path):
#    os.makedirs(feather_path)


data = pd.DataFrame()

count = 0
for file in files:
    if '.file' not in file:
        continue
    try:
        df = feather.read_feather('%s/%s' % (data_path, file))
        df.drop_duplicates(inplace=True)
        if count == 0:
            data = df
        else:
            data = pd.concat([data, df], axis=0)

        count += 1

        print('\n')
        print('\n')
        print('Finished concatenating dataframe for Game ID: ', count)
        print('Completed : ' + str(count) + ' games.')
    except Exception as e:
        print('Error in loading: ' + str(file) + ' file, Error: ' + str(e))

print('\n')
print('\n')
print('Finished concatenating dataframes for all games.')
print(str(count) + ' games counted')

#feather.write_feather(data,'%s/%s.file' % (feather_path, "concat_df"), compression='lz4')





Finished concatenating dataframe for Game ID:  1
Completed : 1 games.




Finished concatenating dataframe for Game ID:  2
Completed : 2 games.




Finished concatenating dataframe for Game ID:  3
Completed : 3 games.




Finished concatenating dataframes for all games.
3 games counted


# Deciding on the GroupBy aggregations to make

In [23]:
data.groupby(['game_id','player_name','EVENTMSGTYPE',]).agg({'spacing_1':'mean', 'spacing_2':'mean'}).sort_values('spacing_2', ascending=False).reset_index()

,game_id,player_name,EVENTMSGTYPE,spacing_1,spacing_2
0,0021500003,Ryan Anderson,12.0,-0.291808,2.885702
1,0021500001,Aron Baynes,12.0,-0.308704,2.417590
2,0021500001,Dennis Schroder,4.0,1.057321,2.055505
3,0021500003,Stephen Curry,4.0,1.127352,1.965201
4,0021500001,Mike Scott,4.0,1.474634,1.963687
...,...,...,...,...,...
334,0021500003,Marreese Speights,4.0,NaN,NaN
335,0021500003,Marreese Speights,8.0,NaN,NaN
336,0021500003,Shaun Livingston,4.0,NaN,NaN
337,0021500003,Shaun Livingston,8.0,NaN,NaN


In [6]:
player_spacing_df = data[(data.EVENTMSGTYPE == 1) | (data.EVENTMSGTYPE == 2)].groupby(['game_id','position', 'player_name', 'team_name', 'EVENTMSGTYPE','player_dist_bin']).agg({'spacing_1':'mean', 'spacing_2': 'mean'}).reset_index()
#player_spacing_df.to_csv('player_spacing_df.csv')


In [28]:
player_spacing_df

,game_id,position,player_name,team_name,EVENTMSGTYPE,player_dist_bin,spacing_1,spacing_2
0,0021500001,C,Andre Drummond,Detroit Pistons,1.0,0.0,-0.218258,0.887796
1,0021500001,C,Andre Drummond,Detroit Pistons,1.0,8.0,0.010852,0.511675
2,0021500001,C,Andre Drummond,Detroit Pistons,1.0,16.0,-0.062418,0.227338
3,0021500001,C,Andre Drummond,Detroit Pistons,1.0,24.0,0.232508,0.303953
4,0021500001,C,Andre Drummond,Detroit Pistons,2.0,0.0,0.116411,1.106956
...,...,...,...,...,...,...,...,...
475,0021500003,G-F,Andre Iguodala,Golden State Warriors,1.0,24.0,0.319776,0.498602
476,0021500003,G-F,Andre Iguodala,Golden State Warriors,2.0,0.0,0.253818,0.799129
477,0021500003,G-F,Andre Iguodala,Golden State Warriors,2.0,8.0,0.293932,0.531105
478,0021500003,G-F,Andre Iguodala,Golden State Warriors,2.0,16.0,0.057097,0.452293


In [14]:
data.groupby(['game_id','position', 'player_name', 'shooter_ind','player_dist_bin']).agg({'spacing_1':'mean', 'spacing_2': 'mean'})

spacing_1  \
game_id    position player_name    shooter_ind player_dist_bin              
0021500001 C        Andre Drummond 0           0.0              -0.424117   
                                               8.0              -0.217525   
                                               16.0             -0.083734   
                                               24.0              0.110431   
                                   1           0.0                    NaN   
...                                                                   ...   
0021500003 G-F      Andre Iguodala 0           24.0              0.442751   
                                   1           0.0                    NaN   
                                               8.0                    NaN   
                                               16.0                   NaN   
                                               24.0                   NaN   

                                                                spacing_2  
game_id    position player_name    shooter_ind player_dist_bin             
0021500001 C        Andre Drummond 0           0.0               0.668253  
                                               8.0               0.448991  
                                               16.0              0.291826  
                                               24.0              0.602412  
                                   1           0.0                    NaN  
...                                                                   ...  
0021500003 G-F      Andre Iguodala 0           24.0              0.696277  
                                   1           0.0                    NaN  
                                               8.0                    NaN  
                                               16.0                   NaN  
                                               24.0                   NaN  

[431 rows x 2 columns]

In [19]:
data[data.player_id != -1].groupby(['team_name']).agg({'spacing_2':'mean'}).sort_values('spacing_2', ascending=False).reset_index()

,team_name,spacing_2
0,Atlanta Hawks,0.669634
1,Detroit Pistons,0.665796
2,Golden State Warriors,0.564249
3,New Orleans Pelicans,0.512738
4,Cleveland Cavaliers,0.453308
5,Chicago Bulls,0.433434
